# Scrapy 실습
> 간단한 Scrapy 실습을 정리하였습니다

- toc: true
- badges: true
- comments: false
- categories: [crawling]
- image:

# Scrapy 실습

http꞉// 를 붙여주면, start_urls 에서 http꞉// 를 무조건 붙여주기 때문에, 결과적으로 http꞉// 가 두 번 붙게되어, 강제로 http꞉// 를 start_urls 에서 삭제해야 한다

``` shell
// 프로젝트를 생성하고 크롤러 모듈을 생성하자

scrapy startproject ecommerce1

scrapy genspider gmarket_best corners.gmarket.co.kr/Bestsellers
```
위의 명령어를 통해 url에 맞는 크롤러 python 모듈을 생성하자

start_urls는 def parse의 response로 들어가게 된다


- 구조를 한번 살펴보고 내려가자
- ** ** 표시를 한 것이 오늘의 핵심 파일이 될 예정이다.


```shell

scrapy.cfg # deploy configuration file
    ecommerce1/ # project's Python module, you'll import your code from her
        __init__.py
        **items.py** # project items definition file
        pipelines.py # project pipelines file
        settings.py # project settings file
        spiders/ # a directory where you'll later put your spiders
            __init__.py
            **gmarket.py**

```

 spiders/gmarket_best.py <br>
``def parse`` 부분을 아래와 같이 변경

In [1]:
import scrapy


class GmarketBestSpider(scrapy.Spider):
    name = 'gmarket_best'
    allowed_domains = ['corners.gmarket.co.kr']
    start_urls = ['http://corners.gmarket.co.kr/']

    def parse(self, response):
        titles = response.css('div.best-list li > a::text').getall()
        for title in titles:
            print(title)

 !scrapy crawl gmarket_best 을 통해 크롤링 수행할 수 있다

### items.py

> 크롤링 데이터 다루기꞉ 저장하기
items.py 파일 확인해보자 > items.py # project items definition file

어떤 아이템들을 가져올건지 items.py에 선언을 해줘야 한다

선언이 된 아이템들을 gmarket_best.py에서 가져와 전달을 해준다.
위의 과정을 거쳐야 scrapy에서 저장을 하는 등의 작업을 수행할 수 있게 된다



- items.py

In [3]:
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class Ecommerce1Item(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    # 저장할 데이터 이름 = scrapy.Field() 과 같이 작성
    title = scrapy.Field()

items.py에 전달하기 위해서는 ``gmarket_best.py``를 아래와 같이 수정해야 한다

#### gmarket_best.py 수정

```python

import scrapy

### items.py 의 클래스명인 Ecommerce1Item 을 import 하기

from ecommerce1.items import Ecommerce1Item

class GmarketSpider(scrapy.Spider):
    name = 'gmarket_best'
    allowed_domains = ['corners.gmarket.co.kr/Bestsellers']
    start_urls = ['http://corners.gmarket.co.kr/Bestsellers/']

    def parse(self, response):
        titles = response.css('div.best-list li > a::text').getall()

        for title in titles:
            item = Ecommerce1Item() #선언을 통해 item 객체 생성
            # items.py 에서 정의한 scrapy.Field() 명을 동일하게 써줘야 함
            item['title'] = title # parsing해서 가져온 데이터를 field 열에 계속 넣어준다.
            yield item # yield 하는 순간 데이터가 items.py로 쌓인다

```
- items.py 필드명 선언 ->
- crawling python file에서 items.py import를 통해 불러와 item 객체 생성 ->
- 미리 생성해놓은 필드에 parsing해온 데이터를 yield를 통해 적재하기


#### 다양한 데이터 format으로 아이템들을 저장할 수 있다.

- csv, xml, json 포멧
- 터미널 환경에서, 크롤링을 실행 시켰던 ecommerce1/ecommerce1 폴더에서 다음 명령을 수행하자

```
// scrapy crawl 크롤러명 -o 저장할 파일명 -t 저장포멧

// 예
scrapy crawl gmarket_best -o gmarket_best.csv -t csv
scrapy crawl gmarket_best -o gmarket_best.xml -t xml

scrapy crawl gmarket_best -o gmarket_best.json -t json
> json 파일을 확인하면, 한글문자가 깨져나온다 > settings.py를 수정해줘야 한다
```

**settings.py**에 들어가서 아래의 코드를 추가해주어야 한다

해당 파일 안에 utf-8 encoding 설정을 추가해준다 (위치는 상관 없다)

```shell
# FEED_EXPORT_ENCODING 추가
FEED_EXPORT_ENCODING = 'utf-8'
```

```shell

! scrapy crawl gmarket_best -o gmarket_best.csv -t csv

// 아래와 같이 gmarket_best.csv 파일이 생성된 것을 확인 할 수 있다.
! ls
__init__.py      **gmarket_best.csv** middlewares.py   settings.py
__pycache__      items.py         pipelines.py     spiders
```
